In [1]:
from copy import deepcopy
import sys
from pprint import pprint

import yaml


sys.path.append('D:/Dropbox/89_other_application/GitHub/PETsARD')

filename = 'DevTest_Exec_Yaml.yaml'

with open(filename, 'r') as yaml_file:
    yaml_raw: dict = yaml.safe_load(yaml_file)

pprint (yaml_raw)

{'Evaluator': {'sd-qlt': {'method': 'sdmetrics-single_table-qualityreport'}},
 'Loader': {'adult': {'filepath': 'benchmark://adult',
                      'na_values': {'native-country': '?',
                                    'occupation': '?',
                                    'workclass': '?'}}},
 'Postprocessor': {'missing-drop': None},
 'Preprocessor': {'missing-drop': {'encoder': {'education': 'encoder_label',
                                               'gender': 'encoder_label',
                                               'income': 'encoder_uniform',
                                               'marital-status': 'encoder_label',
                                               'native-country': 'encoder_uniform',
                                               'occupation': 'encoder_uniform',
                                               'race': 'encoder_uniform',
                                               'relationship': 'encoder_label',
                           

In [2]:
# _splitter_handler

from PETsARD.config import Config

cfg = Config(filename=filename)

pprint (yaml_raw['Splitter'])
pprint (cfg.yaml['Splitter'])

{'0.5': {'num_samples': 2, 'train_split_ratio': 0.5},
 '0.8': {'num_samples': 2, 'train_split_ratio': 0.8}}
{'0.5_[1|2]': {'num_samples': 1, 'train_split_ratio': 0.5},
 '0.5_[2|2]': {'num_samples': 1, 'train_split_ratio': 0.5},
 '0.8_[1|2]': {'num_samples': 1, 'train_split_ratio': 0.8},
 '0.8_[2|2]': {'num_samples': 1, 'train_split_ratio': 0.8}}


In [3]:
from PETsARD.operator import LoaderOperator

loading = LoaderOperator(config=cfg.yaml['Loader']['adult'])
loading.run(input={})
pprint(loading.get_result().head(2))

Loader - Benchmarker: file benchmark\adult.csv already exist and match SHA-256.
                      PETsARD will ignore download and use local data directly.
   age workclass  fnlwgt education  educational-num      marital-status  \
0   25   Private  226802      11th                7       Never-married   
1   38   Private   89814   HS-grad                9  Married-civ-spouse   

          occupation relationship   race gender  capital-gain  capital-loss  \
0  Machine-op-inspct    Own-child  Black   Male             0             0   
1    Farming-fishing      Husband  White   Male             0             0   

   hours-per-week native-country income  
0              40  United-States  <=50K  
1              50  United-States  <=50K  


In [4]:
from PETsARD.operator import SplitterOperator

splitting = SplitterOperator(config=cfg.yaml['Splitter']['0.8_[1|2]'])
splitting.run(input={'data': loading.get_result()})
print('train')
pprint(splitting.get_result()['train'].head(2))
print('validation')
pprint(splitting.get_result()['validation'].head(2))


train
   age  workclass  fnlwgt   education  educational-num      marital-status  \
0   25    Private  226802        11th                7       Never-married   
1   28  Local-gov  336951  Assoc-acdm               12  Married-civ-spouse   

          occupation relationship   race gender  capital-gain  capital-loss  \
0  Machine-op-inspct    Own-child  Black   Male             0             0   
1    Protective-serv      Husband  White   Male             0             0   

   hours-per-week native-country income  
0              40  United-States  <=50K  
1              40  United-States   >50K  
validation
   age workclass  fnlwgt education  educational-num      marital-status  \
0   38   Private   89814   HS-grad                9  Married-civ-spouse   
1   72       NaN  173427   HS-grad                9  Married-civ-spouse   

        occupation relationship   race gender  capital-gain  capital-loss  \
0  Farming-fishing      Husband  White   Male             0             0   
1   

In [5]:
import warnings

from PETsARD.operator import PreprocessorOperator

with warnings.catch_warnings():
    # 忽略 FutureWarning. See Issue # 244: https://github.com/matheme-justyn/PETsARD/issues/244
    warnings.simplefilter(action='ignore', category=FutureWarning)

    preproc = PreprocessorOperator(config=cfg.yaml['Preprocessor']['missing-drop'])
    preproc.run(input={
        'data': splitting.get_result()['train'],
        'metadata': loading.loader.metadata
    })
    pprint(preproc.get_result().head(2))

   age  workclass  fnlwgt  education  educational-num  marital-status  \
0   25   0.657114  226802          1                7               4   
1   28   0.859989  336951          7               12               2   

   occupation  relationship      race  gender  capital-gain  capital-loss  \
0    0.746602             3  0.891961       1             0             0   
1    0.984000             0  0.833341       1             0             0   

   hours-per-week  native-country    income  
0              40        0.138132  0.231523  
1              40        0.744804  0.950319  


In [6]:
from PETsARD.operator import SynthesizerOperator

syn = SynthesizerOperator(config=cfg.yaml['Synthesizer']['sdv-gaussian'])
syn.run(input={'data': preproc.get_result()})
pprint(syn.get_result().head(2))


Synthesizer (SDV - SingleTable): Metafile loading time: 0.0452 sec.
Synthesizer (SDV - SingleTable): Fitting GaussianCopula.
Synthesizer (SDV - SingleTable): Fitting  GaussianCopula spent 14.282 sec.
Synthesizer (SDV - SingleTable): Sampling GaussianCopula # 26631 rows (same as raw) in 1.6842 sec.
   age  workclass  fnlwgt  education  educational-num  marital-status  \
0   55   0.524161  176864         11               10               2   
1   36   0.643725  378978          6               11               4   

   occupation  relationship      race  gender  capital-gain  capital-loss  \
0    0.286894             1  0.360951       1             0             0   
1    0.795163             0  0.811977       1             0             0   

   hours-per-week  native-country    income  
0              36        0.415389  0.982648  
1              42        0.049176  0.723963  


In [7]:
from PETsARD.operator import PostprocessorOperator

postproc = PostprocessorOperator(config=cfg.yaml['Postprocessor']['missing-drop'])
postproc.run(input={
    'preprocessor': preproc.processor,
    'data': syn.get_result()
})
pprint(postproc.get_result().head(2))

   age workclass  fnlwgt education  educational-num      marital-status  \
0   55   Private  176864   HS-grad               10  Married-civ-spouse   
1   36       NaN  378978       9th               11       Never-married   

     occupation   relationship   race gender  capital-gain  capital-loss  \
0  Craft-repair  Not-in-family  White   Male             0             0   
1           NaN        Husband  White   Male             0             0   

   hours-per-week native-country income  
0              36  United-States   >50K  
1              42  United-States  <=50K  


In [11]:
from PETsARD.operator import EvaluatorOperator

evaluating = EvaluatorOperator(config=cfg.yaml['Evaluator']['sd-qlt'])
evaluating.run(input={
    'data': {
        'ori': splitting.get_result()['train'],
        'syn': postproc.get_result(),
        'control': splitting.get_result()['validation'],
    }
})
pprint(evaluating.get_result()['details']['Column Pair Trends'].head(2))

Evaluator (SDMetrics): Evaluating QualityReport.
Generating report ...
(2/2) Evaluating Column Pair Trends: : 100%|██████████| 105/105 [00:06<00:00, 15.55it/s]

Overall Score: 83.93%

Properties:
- Column Shapes: 88.08%
- Column Pair Trends: 79.79%
Evaluator (SDMetrics): Evaluating QualityReport spent 7.0198 sec.
  Column 1   Column 2                 Metric     Score  Real Correlation  \
0      age  workclass  ContingencySimilarity  0.892130               NaN   
1      age     fnlwgt  CorrelationSimilarity  0.995742         -0.079317   

   Synthetic Correlation  
0                    NaN  
1              -0.070801  


In [1]:
from copy import deepcopy
import sys
from pprint import pprint


sys.path.append('D:/Dropbox/89_other_application/GitHub/PETsARD')
filename = 'DevTest_Exec_Yaml.yaml'

from PETsARD.config import Config, Status

cfg = Config(filename=filename)

# pprint([cfg.config.get()      for _ in range(cfg.config.qsize()     )])
# pprint([cfg.module_flow.get() for _ in range(cfg.module_flow.qsize())])
# pprint([cfg.expt_flow.get()   for _ in range(cfg.expt_flow.qsize()  )])

In [2]:
import warnings

with warnings.catch_warnings():
    # 忽略 FutureWarning. See Issue # 244: https://github.com/matheme-justyn/PETsARD/issues/244
    warnings.simplefilter(action='ignore', category=FutureWarning)

    sequence = [
        'Loader', 'Splitter', 'Preprocessor', 'Synthesizer', 'Postprocessor', 'Evaluator'
    ]

    sts: Status = Status(sequence=sequence)
    pre_module: str = ''
    result = {}

    while cfg.config.qsize() > 0:
        operator = cfg.config.get()
        module   = cfg.module_flow.get()
        expt     = cfg.expt_flow.get()
        exclude_index: list = []

        print(f"module {module}")
        
        operator.run(sts.get_input(module=module))

        sts.put(module, expt, operator)

        if module == sequence[-1]:
            full_expt = sts.get_full_expt()
            full_expt_str = '_'.join(
                [f"{module}[{expt}]" for module, expt in full_expt.items()]
            )
            result[full_expt_str] = deepcopy(sts.get_result(module=module))


module Loader
Loader - Benchmarker: file benchmark\adult.csv already exist and match SHA-256.
                      PETsARD will ignore download and use local data directly.
module Splitter
module Preprocessor
module Synthesizer
Synthesizer (SDV - SingleTable): Metafile loading time: 0.0245 sec.
Synthesizer (SDV - SingleTable): Fitting GaussianCopula.
Synthesizer (SDV - SingleTable): Fitting  GaussianCopula spent 12.6798 sec.
Synthesizer (SDV - SingleTable): Sampling GaussianCopula # 26679 rows (same as raw) in 1.626 sec.
module Postprocessor
module Evaluator
Evaluator (SDMetrics): Evaluating QualityReport.
Generating report ...
(2/2) Evaluating Column Pair Trends: : 100%|██████████| 105/105 [00:05<00:00, 18.23it/s]

Overall Score: 79.37%

Properties:
- Column Shapes: 83.03%
- Column Pair Trends: 75.71%
Evaluator (SDMetrics): Evaluating QualityReport spent 6.0281 sec.
module Splitter
module Preprocessor
module Synthesizer
Synthesizer (SDV - SingleTable): Metafile loading time: 0.0278 s

ValueError: Bin edges must be unique: Index([               0.0, 0.9137032559302956, 0.9340476091216076,
       0.9395089006545212, 0.9441780964689207, 0.9483053320548631,
       0.9522241213990912, 0.9553508150248051, 0.9582690624088048,
       0.9611456205444616, 0.9638971109350899, 0.9665652228290325,
       0.9688998207362322, 0.9710676616500605, 0.9731521240672032,
       0.9750698294909744, 0.9769458456664027, 0.9787801725934883,
       0.9805728102722309, 0.9823237587026307,  0.983907950139659,
       0.9853670738316589, 0.9866594405302873, 0.9879518072289157,
       0.9891607954308584, 0.9901613373910869, 0.9911618793513154,
       0.9921624213115439, 0.9931629632717723, 0.9941218159836579,
       0.9948722224538292, 0.9955809396756578, 0.9962896568974863,
       0.9968733063742862, 0.9973735773544004, 0.9978738483345145,
       0.9983741193146287, 0.9988327010464001, 0.9992495935298287,
       0.9996664860132572,                1.0,                1.0],
      dtype='float64').

In [4]:
import pprint
pp = pprint.PrettyPrinter(depth=1)
pp.pprint(result.keys())

dict_keys(['Loader[adult]_Splitter[0.8_[1|2]]_Preprocessor[missing-drop]_Synthesizer[sdv-gaussian]_Postprocessor[missing-drop]_Evaluator[sd-qlt]', 'Loader[adult]_Splitter[0.8_[2|2]]_Preprocessor[missing-drop]_Synthesizer[sdv-gaussian]_Postprocessor[missing-drop]_Evaluator[sd-qlt]'])
